In [78]:
import tensorflow as tf

import numpy as np
import os
import time

### Analyse data

In [79]:
#path_to_file = tf.keras.utils.get_file('./autodabarca')
path_to_file = './autodabarca.txt'

In [80]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')
# Take a look at the first 250 characters in text
print(text[:250])

Length of text: 28747 characters
DIABO
À barca, à barca, venham lá!
Que temos gentil maré!
– dirigindo-se ao companheiro –
Ora põe o barco à ré!
COMPANHEIRO DO DIABO
Está feito, está feito!
DIABO
Bem feito está!
Vai agora, em má hora,
Esticar aquele palanco 
E desocupar a


In [81]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
print(vocab)

80 unique characters
['\n', '\r', ' ', '!', '"', "'", ',', '-', '.', '1', '2', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'z', 'º', 'À', 'Â', 'É', 'Í', 'Ó', 'Ú', 'à', 'á', 'ã', 'ç', 'é', 'ê', 'í', 'ó', 'ô', 'õ', 'ú', '–', '”', '…']


### Prepare data

In [82]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [83]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [84]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Split the text into characters and convert each character into a numeric ID

In [85]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(28747,), dtype=int64, numpy=array([18, 23, 15, ..., 37, 48,  9], dtype=int64)>

In [86]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [87]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

D
I
A
B
O



À
 
b


In [88]:
seq_length = 100

In [89]:
#create batches for training
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'D' b'I' b'A' b'B' b'O' b'\r' b'\n' b'\xc3\x80' b' ' b'b' b'a' b'r' b'c'
 b'a' b',' b' ' b'\xc3\xa0' b' ' b'b' b'a' b'r' b'c' b'a' b',' b' ' b'v'
 b'e' b'n' b'h' b'a' b'm' b' ' b'l' b'\xc3\xa1' b'!' b'\r' b'\n' b'Q' b'u'
 b'e' b' ' b't' b'e' b'm' b'o' b's' b' ' b'g' b'e' b'n' b't' b'i' b'l'
 b' ' b'm' b'a' b'r' b'\xc3\xa9' b'!' b'\r' b'\n' b'\xe2\x80\x93' b' '
 b'd' b'i' b'r' b'i' b'g' b'i' b'n' b'd' b'o' b'-' b's' b'e' b' ' b'a'
 b'o' b' ' b'c' b'o' b'm' b'p' b'a' b'n' b'h' b'e' b'i' b'r' b'o' b' '
 b'\xe2\x80\x93' b'\r' b'\n' b'O' b'r' b'a' b' ' b'p' b'\xc3\xb5' b'e'], shape=(101,), dtype=string)


In [90]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'DIABO\r\n\xc3\x80 barca, \xc3\xa0 barca, venham l\xc3\xa1!\r\nQue temos gentil mar\xc3\xa9!\r\n\xe2\x80\x93 dirigindo-se ao companheiro \xe2\x80\x93\r\nOra p\xc3\xb5e'
b' o barco \xc3\xa0 r\xc3\xa9!\r\nCOMPANHEIRO DO DIABO\r\nEst\xc3\xa1 feito, est\xc3\xa1 feito!\r\nDIABO\r\nBem feito est\xc3\xa1!\r\nVai agora, em '
b'm\xc3\xa1 hora,\r\nEsticar aquele palanco \r\nE desocupar aquele banco,\r\nPara a gente que vir\xc3\xa1.\r\n\xe2\x80\x93 falando para '
b'o ar \xe2\x80\x93\r\n\xc3\x80 barca, \xc3\xa0 barca, hu-u!\r\nDepressinha, que se quer ir!\r\nOh, que tempo para partir,\r\nLouvores a'
b' Belzebu!\r\n\xe2\x80\x93 dirigindo-se ao companheiro \xe2\x80\x93\r\nMas ent\xc3\xa3o! que fazes tu?\r\nLimpa todo aquele leito! \r\nCOMP'


Takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [91]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [92]:
dataset = sequences.map(split_input_target)

### Create training batches

In [93]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build The Model

In [94]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [95]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [96]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [97]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 81) # (batch_size, sequence_length, vocab_size)


In [98]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  20736     
_________________________________________________________________
gru_3 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_3 (Dense)              multiple                  83025     
Total params: 4,042,065
Trainable params: 4,042,065
Non-trainable params: 0
_________________________________________________________________


In [99]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([41, 78, 65, 55, 26, 22,  9, 49, 29, 68, 61, 66,  5, 67, 38, 15, 62,
       63, 46,  1, 76, 76,  8, 48, 26, 23, 42, 17, 72, 67, 69, 76, 15, 40,
       28, 68, 31, 80, 64, 10, 26, 34, 31, 79, 55, 47, 76, 73, 75, 15, 56,
       24, 43,  1,  2, 27, 44, 25, 41, 55, 70, 80, 53, 63,  9, 17, 39,  2,
       15, 60, 66, 32,  3,  4, 26, 73, 28, 72, 65, 49, 72, 40, 35, 50, 53,
       21, 51, 16, 76, 13, 39,  2, 71, 26, 80, 33, 19,  4, 29, 70],
      dtype=int64)

Text generated by the model before training:

In [100]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'de \xc3\xa9 a passagem?\r\nDIABO\r\nPara a infernal comarca.\r\nONZENEIRO\r\nDix! N\xc3\xa3o vou eu em tal barca.\r\nAquela '

Next Char Predictions:
 b'e\xe2\x80\x93\xc3\x93tMH.nP\xc3\xa1\xc3\x80\xc3\x9a"\xc3\xa0bA\xc3\x82\xc3\x89j\n\xc3\xb5\xc3\xb5-mMIfC\xc3\xaa\xc3\xa0\xc3\xa3\xc3\xb5AdO\xc3\xa1R\xe2\x80\xa6\xc3\x8d1MUR\xe2\x80\x9dtl\xc3\xb5\xc3\xad\xc3\xb4AuJg\n\rNhLet\xc3\xa7\xe2\x80\xa6r\xc3\x89.Cc\rA\xc2\xba\xc3\x9aS !M\xc3\xadO\xc3\xaa\xc3\x93n\xc3\xaadVorGpB\xc3\xb5;c\r\xc3\xa9M\xe2\x80\xa6TE!P\xc3\xa7'


### Train the model

In [101]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [102]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 81)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.3944597, shape=(), dtype=float32)


In [103]:
tf.exp(example_batch_mean_loss).numpy()

81.000854

In [104]:
model.compile(optimizer='adam', loss=loss)

In [105]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [106]:
EPOCHS = 150

In [107]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/150
4/4 [==============================] - 4s 97ms/step - loss: 4.3016
Epoch 2/150
4/4 [==============================] - 0s 88ms/step - loss: 4.4570
Epoch 3/150
4/4 [==============================] - 0s 87ms/step - loss: 4.1529
Epoch 4/150
4/4 [==============================] - 0s 88ms/step - loss: 4.0240
Epoch 5/150
4/4 [==============================] - 0s 88ms/step - loss: 3.7411
Epoch 6/150
4/4 [==============================] - 1s 91ms/step - loss: 3.4959
Epoch 7/150
4/4 [==============================] - 0s 87ms/step - loss: 3.2952
Epoch 8/150
4/4 [==============================] - 0s 92ms/step - loss: 3.2013
Epoch 9/150
4/4 [==============================] - 0s 88ms/step - loss: 3.0607
Epoch 10/150
4/4 [==============================] - 0s 86ms/step - loss: 2.9547
Epoch 11/150
4/4 [==============================] - 0s 87ms/step - loss: 2.8312
Epoch 12/150
4/4 [==============================] - 0s 88ms/step - loss: 2.7137
Epoch 13/150
4/4 [==============================]

#### Generate text

In [108]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [109]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [110]:
start = time.time()
states = None
next_char = tf.constant(['DIABO'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

DIABO
À barca, lugo no!
PARVO
Ao Inferno? Pentanfo sada.
Vol tanta terra alinha,
A pensar nos eupornfado
Que foi da tua sapitos?
A telheras sou perio.
–NZENou no que era el.
SAPATEIRO
Ó da barca!
DIABO
Que quereis?
CORREGEDOR
Está aqui o senhor juiz?
DIABO
Oh amarquem me digais.
JUDEU
Eis aqui quatro tostões,
E maiso parre inforca.
Oh alas entrará isso bém,
Nom toPATEIROS
E osfria do mou ei entonde,
Que ninguem me deixais.
CAVanho de cortrna o entanca! Oh! Qua tão está
Leva quem rouba de praça.
Oh almas embaraçás,
Sou sem rodo em mais alguém,
Merecedor de tal bem,
Que deva entrar aqui.
Vem ambarcai e partiremos:
Tomareis um par drízoda
Com ou tocisso, seu fado no batel do Anjo io betel!
ANJO
Oh! haboatas lovar de Ansento!
Peltu de vós prazeres.
CORREGEDOR
E onde vai o batel?
DIABO
No Inão nogatais esta cá!
Mai quem andenadoeres de ir.
ONZENEIRO
E vamos já partir?
DIABO
Não penses em mais delença.
FIDALGO
E porqui, de ouvi?...
– dirigindo-se ao  

_______________________________________

### Export the generator

In [111]:
tf.saved_model.save(one_step_model, 'one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


Retrieve the generator

In [112]:
one_step_reloaded = tf.saved_model.load('one_step')

In [113]:
states = None
next_char = tf.constant(['DIABO'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

DIABO
Itborquetiti, que Barre tão senvor.
ANJO
Atará cá muito que levevis
Hei a pão que te caiu!
A m
